In [ ]:
import os
import sys
import platform

# Detect environment
def detect_environment():
    """Detect if running in Google Colab or local environment"""
    try:
        import google.colab
        return "colab"
    except ImportError:
        return "local"

ENV = detect_environment()

print("=" * 60)
print("DEPTH PRO - UNIVERSAL ENVIRONMENT")
print("=" * 60)
print(f"Environment: {ENV.upper()}")
print(f"Platform: {platform.system()} {platform.machine()}")
print(f"Python: {sys.version}")

# Check GPU availability
try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Check for Apple Silicon (MPS)
    if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        print("Apple Silicon GPU (MPS) available")
except ImportError:
    print("PyTorch not installed - will install in next step")

print(f"Current working directory: {os.getcwd()}")
print("=" * 60)


In [ ]:
# Install dependencies
print("Installing dependencies...")
import subprocess

packages = [
    'opencv-python-headless',
    'timm',
    'matplotlib',
    'tqdm',
    'ipywidgets'  # For interactive file upload widget
]

if ENV == "colab":
    # Google Colab - install everything needed
    for package in packages:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
else:
    # Local environment - install only what's missing
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
        except subprocess.CalledProcessError:
            print(f"Warning: Could not install {package}")

print("Dependencies installed!")


In [ ]:
import os
import sys

if ENV == "colab":
    # In Colab, clone the repository
    if not os.path.exists('ml-depth-pro'):
        print("Cloning Apple's Depth Pro repository...")
        os.system("git clone https://github.com/apple/ml-depth-pro.git")
    else:
        print("Repository already exists")
    
    # Change to the repository directory
    os.chdir('ml-depth-pro')
    print(f"Current directory: {os.getcwd()}")
else:
    # For local Jupyter, assume we're already in the right directory
    if os.path.exists('src/depth_pro'):
        print("Found depth_pro source directory")
    else:
        print("Warning: depth_pro source not found.")
        print("Make sure you're in the ml-depth-pro directory or:")
        print("Run: git clone https://github.com/apple/ml-depth-pro.git && cd ml-depth-pro")

# Add to Python path
sys.path.insert(0, './src')
sys.path.insert(0, '.')

print("Repository setup complete!")


In [ ]:
import os
import urllib.request
from tqdm import tqdm

def download_with_progress(url, filename):
    """Download file with progress bar"""
    class TqdmUpTo(tqdm):
        def update_to(self, b=1, bsize=1, tsize=None):
            if tsize is not None:
                self.total = tsize
            self.update(b * bsize - self.n)
    
    with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=filename) as t:
        urllib.request.urlretrieve(url, filename, reporthook=t.update_to)

# Download model if not already present
model_path = 'depth_pro.pt'
if not os.path.exists(model_path):
    print("Downloading Depth Pro model weights (1.8GB)...")
    print("This may take a few minutes depending on your connection...")
    
    model_url = 'https://huggingface.co/apple/depth-pro/resolve/main/depth_pro.pt'
    download_with_progress(model_url, model_path)
    
    print(f"Model downloaded! Size: {os.path.getsize(model_path) / 1e9:.1f} GB")
else:
    print(f"Model already exists! Size: {os.path.getsize(model_path) / 1e9:.1f} GB")


In [ ]:
# Install the depth_pro package
print("Installing Depth Pro package...")
import subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-e', '.'])

# Test the installation
try:
    import depth_pro
    print("Depth Pro imported successfully!")
    MODEL_AVAILABLE = True
except ImportError as e:
    print(f"Import failed: {e}")
    print("Trying alternative import method...")
    
    # Alternative: add all necessary paths
    import sys
    sys.path.append('./src')
    sys.path.append('./src/depth_pro')
    
    try:
        import depth_pro
        print("Alternative import successful!")
        MODEL_AVAILABLE = True
    except ImportError as e2:
        print(f"Still failed: {e2}")
        MODEL_AVAILABLE = False


In [ ]:
import torch
import numpy as np
from PIL import Image
import cv2
import time
import warnings
warnings.filterwarnings('ignore')

# Device selection based on environment
def select_device():
    """Select the best available device"""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        device_name = f"CUDA GPU ({torch.cuda.get_device_name(0)})"
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        device_name = "Apple Silicon GPU (MPS)"
    else:
        device = torch.device("cpu")
        device_name = "CPU"
    
    print(f"Using device: {device_name}")
    return device, device_name

device, device_name = select_device()

# Global model variables
model = None
transform = None
load_time = 0.0

def load_model():
    """Load the Depth Pro model once"""
    global model, transform, load_time
    if model is None and MODEL_AVAILABLE:
        print("Loading Depth Pro model...")
        start_time = time.time()
        
        try:
            # Load model and transform
            model, transform = depth_pro.create_model_and_transforms()
            
            # Move to appropriate device
            model = model.to(device)
            model.eval()
            
            load_time = time.time() - start_time
            print(f"Model loaded successfully in {load_time:.1f}s!")
            print(f"Model moved to: {device}")
            return True
        except Exception as e:
            print(f"Error loading model: {e}")
            return False
    return model is not None

print("Processing functions defined successfully!")


In [ ]:
def process_image(image_path):
    """Process uploaded image and display results"""
    if not MODEL_AVAILABLE:
        print("❌ Depth Pro model not available. Please check installation.")
        return
    
    try:
        # Load model if not already loaded
        if not load_model():
            print("❌ Failed to load model")
            return
        
        # Load and process image
        if isinstance(image_path, str):
            image = Image.open(image_path)
        else:
            image = image_path
            
        # Convert PIL image to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"📸 Processing image: {image.size}")
        start_time = time.time()
        
        # Use depth_pro's preprocessing
        processed_image = transform(image)
        
        # Add batch dimension and move to device
        if processed_image.dim() == 3:
            processed_image = processed_image.unsqueeze(0)
        
        processed_image = processed_image.to(device)
        
        # Run inference
        with torch.no_grad():
            if device.type == "mps":
                # Use CPU autocast for MPS to avoid potential issues
                with torch.autocast(device_type="cpu", dtype=torch.float16):
                    prediction = model.infer(processed_image)
            else:
                prediction = model.infer(processed_image)
        
        depth = prediction["depth"].cpu().numpy().squeeze()
        focal_length = prediction["focallength_px"]
        
        inference_time = time.time() - start_time
        
        # Normalize depth map for visualization (0-255)
        depth_min, depth_max = depth.min(), depth.max()
        depth_normalized = ((depth - depth_min) / (depth_max - depth_min) * 255).astype(np.uint8)
        
        # Create colormap version
        depth_colored = cv2.applyColorMap(depth_normalized, cv2.COLORMAP_PLASMA)
        depth_colored = cv2.cvtColor(depth_colored, cv2.COLOR_BGR2RGB)
        
        # Display results
        import matplotlib.pyplot as plt
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Original image
        axes[0].imshow(image)
        axes[0].set_title('📷 Original Image', fontsize=14, fontweight='bold')
        axes[0].axis('off')
        
        # Colored depth map
        axes[1].imshow(depth_colored)
        axes[1].set_title('🌈 Colored Depth Map', fontsize=14, fontweight='bold')
        axes[1].axis('off')
        
        # Grayscale depth map
        axes[2].imshow(depth_normalized, cmap='gray')
        axes[2].set_title('⚫ Grayscale Depth Map', fontsize=14, fontweight='bold')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Print performance metrics
        print("\n" + "="*60)
        print("🎯 DEPTH MAP GENERATION COMPLETE!")
        print("="*60)
        print(f"🌍 Environment: {ENV.upper()}")
        print(f"⚡ Device: {device_name}")
        print(f"📐 Image Size: {image.width} × {image.height}")
        print(f"🔍 Estimated Focal Length: {focal_length:.1f} pixels")
        print(f"📏 Depth Range: {depth_min:.2f}m - {depth_max:.2f}m")
        print(f"⏱️  Processing Time: {inference_time:.3f}s")
        print(f"🚀 Model Load Time: {load_time:.1f}s (one-time)")
        print(f"🤖 Model: Apple Depth Pro v1.0")
        print("="*60)
        print("💡 How to use the results:")
        print("   • Colored Version: Great for visualization and analysis")
        print("   • Grayscale Version: Use for 3D reconstruction, depth-based effects")
        print("   • Depth Values: White = closer, Black = farther")
        
    except Exception as e:
        print(f"❌ Error processing image: {str(e)}")
        import traceback
        traceback.print_exc()

print("Image processing function defined successfully!")


In [ ]:
from ipywidgets import FileUpload, Button, Output, VBox, HTML
from IPython.display import display, clear_output
import io

# Create upload widget
print("📁 Setting up file upload interface...")

upload_widget = FileUpload(
    accept='image/*',  # Accept only image files
    multiple=False,    # Single file upload
    description='Choose Image',
    button_style='primary'
)

process_button = Button(
    description='🚀 Generate Depth Map',
    button_style='success',
    layout={'width': '200px', 'height': '40px'}
)

output_area = Output()

def on_upload_change(change):
    """Handle file upload"""
    clear_output()
    if upload_widget.value:
        filename = list(upload_widget.value.keys())[0]
        print(f"📸 Image uploaded: {filename}")
        print("✅ Ready to process! Click the button below to generate depth map.")
    else:
        print("📁 Please upload an image file (JPG, PNG, etc.)")

def on_process_click(button):
    """Handle process button click"""
    with output_area:
        clear_output()
        
        if not upload_widget.value:
            print("❌ Please upload an image first!")
            return
            
        # Get the uploaded file
        filename = list(upload_widget.value.keys())[0]
        file_content = upload_widget.value[filename]['content']
        
        # Convert to PIL Image
        image = Image.open(io.BytesIO(file_content))
        print(f"📁 Processing uploaded file: {filename}")
        
        # Process the image
        process_image(image)

# Set up event handlers
upload_widget.observe(on_upload_change, names='value')
process_button.on_click(on_process_click)

print("✅ Upload interface ready!")


In [ ]:
# Display the upload interface
if MODEL_AVAILABLE:
    print("\n" + "="*60)
    print("🎯 DEPTH PRO - READY TO PROCESS IMAGES!")
    print("="*60)
    print(f"🌍 Environment: {ENV.upper()}")
    print(f"⚡ Device: {device_name}")
    print(f"🤖 Model Status: {'✓ Loaded' if model else '⏳ Will load on first use'}")
    print("="*60)
    print("📁 Upload an image below to generate its depth map!")
    print("💡 Supports: JPG, PNG, and other common image formats")
    print("🚀 Processing happens instantly in this notebook!")
    
    # Display the upload widget and button
    display(HTML("<h3 style='color: #4CAF50;'>📁 Step 1: Choose an Image File</h3>"))
    display(upload_widget)
    
    display(HTML("<h3 style='color: #2196F3;'>🚀 Step 2: Generate Depth Map</h3>"))
    display(process_button)
    
    display(HTML("<h3 style='color: #FF9800;'>📊 Results will appear below:</h3>"))
    display(output_area)
    
    print("\n✨ Interface is ready!")
    print("👆 Click 'Choose Image' above to get started!")
    
else:
    print("❌ Cannot create interface - Depth Pro model not available")
    print("Please check the installation steps above and ensure all dependencies are properly installed.")
